In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

In [6]:
df1 = pd.read_csv("Data/PreProcessedData.csv")
df_no_plot = df1.drop('Plot', axis = 1)
df2 = pd.read_csv("Data/LDA_topics.csv") #UPDATE THIS TO BE WHATEVER DATA WE GIVE IT
df_LDATOPICS = pd.concat([df_no_plot, df2], axis=1)

In [ ]:
df2 = pd.read("Data/LDA_topics_synonym.csv")
df_LDATOPICS_synonym = pd.concat([df_no_plot, df2], axis=1)

In [5]:
def test_trainsets(df):
    rating = df['IMDbRating']
    xdf = df.drop('IMDbRating', axis=1, inplace=False)
    
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(xdf, rating, test_size=0.2, random_state=42)
    
    X_train_df = pd.DataFrame(X_train)
    X_test_df = pd.DataFrame(X_test)
    y_train_df = pd.DataFrame(y_train)
    y_test_df = pd.DataFrame(y_test)

    return X_train, y_train, X_test, y_test

      IMDbRating          Title    Year  Action  Adult  Adventure  Animation  \
0            7.1       Aberdeen  2000.0     0.0    0.0        0.0        0.0   
1            4.1     The Asylum  2000.0     0.0    0.0        0.0        0.0   
2            6.6      The Beach  2000.0     0.0    0.0        1.0        0.0   
3            5.6  Beautiful Joe  2000.0     0.0    0.0        0.0        0.0   
4            7.7   Billy Elliot  2000.0     0.0    0.0        0.0        0.0   
...          ...            ...     ...     ...    ...        ...        ...   
6727         NaN            NaN     NaN     NaN    NaN        NaN        NaN   
6728         NaN            NaN     NaN     NaN    NaN        NaN        NaN   
6729         NaN            NaN     NaN     NaN    NaN        NaN        NaN   
6730         NaN            NaN     NaN     NaN    NaN        NaN        NaN   
6731         NaN            NaN     NaN     NaN    NaN        NaN        NaN   

      Biography  Comedy  Crime  ...  Un

KeyError: 'IMDbRating'

In [ ]:
#ROOM FOR PARAMETER TESTING

In [ ]:
X_train, y_train, X_test, y_test = test_trainsets(df_LDATOPICS)

# Create the XGBoost model
xgb_model = xgb.XGBRegressor()

# Kfold
kf = KFold(n_splits=5, shuffle=True, random_state=123)

# store the score for each fold
scores = []

mae_scores = []
baseline_scores = []
# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Split the data into training and validation sets
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Train the model on the training data
    xgb_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation data
    y_pred_fold = xgb_model.predict(X_val_fold)
    # Compute the MAE score
    baseline = mean_absolute_error(y_val_fold, np.array([np.mean(y_train['IMDbRating'])] * len(y_pred_fold)))
    mae = mean_absolute_error(y_val_fold, y_pred_fold)
    print(mae)
    print(baseline)
    mae_scores.append(mae)
    baseline_scores.append(baseline)

# Compute the average MAE score
average_mae = sum(mae_scores) / len(mae_scores)
average_baseline = sum(baseline_scores) / len(baseline_scores)

y_pred = xgb_model.predict(X_test)
print(y_pred)
print(y_test)
results = pd.DataFrame({'Actual': y_test['IMDbRating'], 'Prediction': y_pred})
results.to_csv('LDA_Topics_results.csv', index=False)

In [ ]:
X_train, y_train, X_test, y_test = test_trainsets(df_LDATOPICS_synonym)

# Create the XGBoost model
xgb_model = xgb.XGBRegressor()

# Kfold
kf = KFold(n_splits=5, shuffle=True, random_state=123)

# store the score for each fold
scores = []

mae_scores = []
baseline_scores = []
# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Split the data into training and validation sets
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Train the model on the training data
    xgb_model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation data
    y_pred_fold = xgb_model.predict(X_val_fold)
    # Compute the MAE score
    baseline = mean_absolute_error(y_val_fold, np.array([np.mean(y_train['IMDbRating'])] * len(y_pred_fold)))
    mae = mean_absolute_error(y_val_fold, y_pred_fold)
    print(mae)
    print(baseline)
    mae_scores.append(mae)
    baseline_scores.append(baseline)

# Compute the average MAE score
average_mae = sum(mae_scores) / len(mae_scores)
average_baseline = sum(baseline_scores) / len(baseline_scores)

y_pred = xgb_model.predict(X_test)
print(y_pred)
print(y_test)
results = pd.DataFrame({'Actual': y_test['IMDbRating'], 'Prediction': y_pred})
results.to_csv('LDA_Topics_synonym_results.csv', index=False)